In [25]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import torch
import random

In [26]:
from keras.models import Sequential
from keras.layers import Dense, Activation

#
model = Sequential()
model.add(Dense(30, input_dim=784, activation = 'sigmoid'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='sgd',
              loss='mse',
              metrics=['accuracy'])

In [27]:




max_moves = 30  #how many inverse operations on the final destination
goal_position = torch.tensor([[1, 0], [0, 1]], dtype=torch.float64)

def data_generator(matrix_T, matrix_U, num_of_datapoints):
    inverse_T = torch.inverse(matrix_T)
    inverse_U = torch.inverse(matrix_U)
    data = torch.empty((num_of_datapoints, 5), dtype=torch.float64)
    for i in range(num_of_datapoints):
        start = goal_position
        moves = random.randint(1, max_moves)
        coin = 3
        for q in range(moves):
            coin = random.randint(0, 1)
            if coin == 1:
                #  print(start, inverse_U, "=")
                start = torch.matmul(inverse_U, start)
            #  print(start)
            else:
                #  print(start, inverse_T, "=")
                start = torch.matmul(inverse_T, start)
                if q != moves - 1:
                    start = torch.matmul(inverse_U, start)
            # print(start)
        # print("done")
        data[i, 0] = start[0][0]
        data[i, 1] = start[0][1]
        data[i, 2] = start[1][0]
        data[i, 3] = start[1][1]
        data[i, 4] = coin
    return data

#https://stackoverflow.com/questions/36158058/torch-save-tensor-to-csv-file#:~:text=For%20simple%20tables%2C%20you%20can,then%20to%20a%20Pandas%20dataframe.&text=You%20can%20first%20convert%20the,table%20as%20a%20csv%20file.

matrix_T = torch.tensor([[1, 3], [0, 1]], dtype=torch.float64)
matrix_U = torch.tensor([[1, 0], [3, 1]], dtype=torch.float64)

data = data_generator(matrix_T, matrix_U, 10000)
data_table = data.numpy()

x_train = data_table[:, :-1]
y_train = data_table[:, -1]

# matrix_T = torch.tensor([[0, -1], [1, 0]], dtype=torch.float64)
# matrix_U = torch.tensor([[1, 1], [0, 1]], dtype=torch.float64)

data = data_generator(matrix_T, matrix_U, 10000)
data_table = data.numpy()

x_test = data_table[:, :-1]
y_test = data_table[:, -1]


In [28]:
from keras.models import Sequential
from keras.layers import Dense, Activation

#
model = Sequential()
model.add(Dense(128, input_dim=4, activation = 'sigmoid'))
model.add(Dense(64,  activation = 'sigmoid'))
model.add(Dense(16, activation = 'sigmoid'))
model.add(Dense(1, activation = 'sigmoid'))


model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

In [29]:
model.fit(x_train, y_train, epochs=100, batch_size=100)

Epoch 1/100
100/100 [==============================] - 1s 2ms/step - loss: 0.0953 - accuracy: 0.9615
Epoch 2/100
100/100 [==============================] - 0s 2ms/step - loss: 0.0088 - accuracy: 1.0000
Epoch 3/100
100/100 [==============================] - 0s 3ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 4/100
100/100 [==============================] - 0s 3ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 5/100
100/100 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 6/100
100/100 [==============================] - 0s 2ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 7/100
100/100 [==============================] - 0s 2ms/step - loss: 9.5067e-04 - accuracy: 1.0000
Epoch 8/100
100/100 [==============================] - 0s 2ms/step - loss: 7.6352e-04 - accuracy: 1.0000
Epoch 9/100
100/100 [==============================] - 0s 3ms/step - loss: 6.2813e-04 - accuracy: 1.0000
Epoch 10/100
100/100 [==============================] - 0s 2ms/step - loss: 5.2

In [30]:
score = model.evaluate(x_test, y_test, batch_size=100)
print(score[0])
print(score[1])

100/100 [==============================] - 0s 2ms/step - loss: 1.3415e-06 - accuracy: 1.0000
1.3414522754828795e-06
1.0


In [31]:
model.save('full_model.h5')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
model.predict(torch.tensor([[1, 0, 6, 1]]).numpy())

1/1 [==============================] - 0s 87ms/step


array([[0.00119146]], dtype=float32)

In [33]:
# FOLLOW IT THROUGH

matrix_T = torch.tensor([[0, -1], [1, 0]], dtype=torch.float64)
matrix_U = torch.tensor([[1, 1], [0, 1]], dtype=torch.float64)

array = data_generator(matrix_T, matrix_U, 1).numpy()[:, :-1]

matrix = array.reshape(2, 2)
iterations = 0

print(array)
print(matrix)

def is_target_array(array):
    target = np.array([1, 0, 0, 1])
    return np.array_equal(array, target)

while not is_target_array(array) and iterations <= 20:
    inverse_matrix_T = torch.inverse(matrix_T)
    inverse_matrix_U = torch.inverse(matrix_U)
    iterations = iterations + 1
    if model.predict(array) >= 0.5: 
        matrix = matrix @ inverse_matrix_T.numpy()
        print("APPLIED T")
    else: 
        matrix = matrix @ inverse_matrix_U.numpy()
        print("APPLIED U")
    array = matrix.reshape(1,4)

print(array)
print(matrix)

[[  5. -12.  -2.   5.]]
[[  5. -12.]
 [ -2.   5.]]
1/1 [==============================] - 0s 79ms/step
APPLIED U
1/1 [==============================] - 0s 21ms/step
APPLIED U
1/1 [==============================] - 0s 21ms/step
APPLIED U
1/1 [==============================] - 0s 20ms/step
APPLIED U
1/1 [==============================] - 0s 20ms/step
APPLIED U
1/1 [==============================] - 0s 21ms/step
APPLIED U
1/1 [==============================] - 0s 21ms/step
APPLIED U
1/1 [==============================] - 0s 21ms/step
APPLIED U
1/1 [==============================] - 0s 21ms/step
APPLIED U
1/1 [==============================] - 0s 20ms/step
APPLIED U
1/1 [==============================] - 0s 21ms/step
APPLIED U
1/1 [==============================] - 0s 21ms/step
APPLIED U
1/1 [==============================] - 0s 21ms/step
APPLIED U
1/1 [==============================] - 0s 20ms/step
APPLIED U
1/1 [==============================] - 0s 21ms/step
APPLIED U
1/1 [==============